# Tracking a wolf - Calculating distances and speed

Today we are going to work again with the wolves data from the University of Minnesota Twin Cities (https://www.voyageurswolfproject.org). Remember, the voyageur wolf collaboration generously accepted to share the data with us, but they ask that you do not share the data outside of this class as it could be used to harm the wolves.

Here is the table of contents:

1. Loading and mapping one wolf (as we did in week7 class).
2. Computing and plotting distances traveled by the wolf (Task 1a and 1b).
3. Dealing with dates and times to obtain corrected distances and plots.
4. Wolf Distance statistics -summary per day, month and hour (Tasks 2-5).
5. Calculating traveling speed to check how fast the wolf moves (Task 6).

## 1. Loading and mapping one wolf

Data files are located in the wolf_data folder. Each file corresponds to a different wolf. They're all csv files with the column names in the first row.

### 1.1 Import modules and data 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import mplleaflet

In [ ]:
data  = pd.read_csv('wolf_data/V062.csv') #Read only wolf V062
display(data)

# We're going to use the longitude and latitude columns a lot. Let's give them shorter names.
Lo = data['Longitude'] #select only longitude column and name it "Lo"
La = data['Latitude'] #select only latitude column and name it "La"

### 1.2 Plot Wolf V062 trajectory
Remember wolf location was recorded every 20 minutes for 6 months.

In [ ]:
plt.figure(figsize=(6,6)) 
plt.plot(Lo,La)
mplleaflet.display()

## 2. Computing and plotting distances traveled by the wolf

Computing the distance between two points given their GPS coordinates is far from easy. Fortunately, the work has already been done. The module `gpxpy`, submodule `geo`, contains a `distance` function that takes the longitude, latitude, and altitude of each point and returns the distance between them in meters. 

### 2.1 Installing 'gpxpy' module

The 'gpxpy' module we'll need today does not come preinstalled with python. We need to install it ourselves. 

For mac users:
Open the terminal.
Type and execute "conda install -y -c conda-forge gpxpy".

For windows users: 
In the winpython folder, start "WinPython Command Prompt". A terminal opens.
In the terminal, type and execute "pip install gpxpy".

Note: Depending how you installed python you may already have the module. One way to check is to try to import it (in a notebook, execute "import [name of the module]"). If you don't get an error then the module is alreaday instaled. Either way, you can run the commands above. If a module is already installed they'll notice and move on.

In [ ]:
#Import the submodule 'geo' from the module 'gpxpy'
import gpxpy.geo 

### 2.2. Define a custom distance function that ignores altitudes 
To make things a little easier we will define a function that ignores altitudes (which we don't have anyway) and returns the distance in miles:

In [ ]:
#New distance function includes latitude and longitude of 2 GPS points

def distance(lo1,la1,lo2,la2):      
    ''' 
    Distance between two points in miles. 
    
    Arguments:
      lo1: Longitude of point 1.
      la1: Latitude of point 1.
      lo2: Longitude of point 2.
      la2: Latitude of point 2. 
    '''
    return gpxpy.geo.distance(lo1,la1,None,lo2,la2,None)/1609  
#The function will return the distances, while ignoring altitude. The 1609 is the xxxxx

### 2.3. Compute distances using the new function

Now that you have a custome distance function, you can use it to compute distances between the GPS points (latitude and longitude) from the points of the wolf track for wolf V062.

<div class="alert alert-block alert-danger">
<b>Task 1a:</b>
Compute the distance between the first and second points of the track.
</div>

<div class="alert alert-block alert-danger">
<b>Task 1b:</b>
Build a list of the distance traveled by wolf V062 since the last point for every point of the track. The first point doesn't have a previous point; just set its distance to zero. Call it `distance_list`.
</div>

*Hint: Loop over the indices of `Lo`, which are also the indices of `La`, using `range` and `len`.*

For future ease of use we now add a new column called `distance` to the data table with the information we just obtained:

In [ ]:
data['distance'] = distance_list
display(data)
# If you did task 1b correctly, the table should now have an extra 
# "distance" column all the way to the right.

<div class="alert alert-block alert-danger">
<b>Task 1c:</b>
Use the new `distance` column to compute the total distance traveled over the entire data set (about 6 months) and the mean distance traveled between two consecutive points.
</div>

One issue with the mean is that the time elapsed between two consecutive point is not entirely consistent. Most of the time it's 20 minutes, but sometimes it's not. As a result the mean is not very meaningful. We'll get back to this in section 5.

### 2.4. Plot the wolf daily mileage

Now that you have calculated the distance, you can plot it to see how much a wolf moves in a day.

##### Daily mileage histogram
First we will plot the wolf distance mileage as an histogram.

We've talked about statistical properties like the mean, standard deviation, etc. But with this much data, we can reconstruct the whole distribution of distance traveled in a day by plotting a histogram. The pyplot's hist finds the maximum and minimum of the data, divides it in however many equal bins you specify with the bins option, and counts the number of days on which the distance traveled fell in each bin.

In [ ]:
# Compute the distance traveled each day.
distance_by_date = data.groupby('date')['distance'].sum() #use the groupby function to select only the distance per day
# display(distance_by_date)

#Make the histogram
plt.hist(distance_by_date,bins=15, edgecolor='blue', linewidth=1.2, color='c')
plt.xlabel('Number of Days')
plt.ylabel('Distance traveled (miles)')
plt.show()

##### Line plot of wolf daily milage 

In [ ]:
#Make a line plot to see distance per day

plt.plot(distance_by_date.index,distance_by_date, color='b')
plt.xticks(rotation=90)
plt.xlabel('Date')
plt.ylabel('Distance traveled')
plt.show()

#### Some problems with the data and plot:

##### Plot Problem:
The reason why the horizontal axis is so crowded in the plot is because pyplot doesn't recognize the dates as dates, to it they're just a bunch of strings. It's able to tell either when there's a gap (a few days without data).

##### Calculated distance Problem:
Another big problem with the mean we computed is that the time elapsed between two consecutive points on the track is not always super consistent. Most of the time it's 20 minutes, but sometimes the signal is lost for a while and it's a lot longer. 

To address both problems, and compute things like the speed, we need a better way to handle the date and time at which each point was recorded. For this we will use python's datetime objects.


## 3. Proper handling of dates and times

Our data contains the date and time each point was recorded, however they're provided to us as a pair of strings. That's good enough to look at them, but it makes it very difficult to answer questions like "how many miles were traveled during the night?" Or "How fast did the wolf move between those two points?". Both are valuable questions in analyzing the wolf's motion patterns. It's also the reason why the tick labels on the horizontal axis of our daily distance vs date graph above are so unreadable. Pyplot doesn't realize the horizontal axis represents time, it merely treats each date as a random name.

All of these issues can be addressed by converting our date and time strings to a python "datetime" object.

### 3.1. Check your date and time data

First check your date and time data and try to do some arithmetics.

In [ ]:
# Print the first two rows of the table.
display(data.iloc[:2,:])

In [ ]:
# The values in the "time" column are strings.
# We can add them like strings, i.e., concatenate them.
print(data['time'][1]+' and '+data['time'][0])

In [ ]:
# However we cannot perform any "time arithmetic" with them.
# For example, getting the time elapsed between the two.
print(data['time'][1]-data['time'][0])

# The same is true for the "date" column.

### 3.2. Converting our date and time strings to a python 'datetime' object

Python has a `datetime` module with lots of useful functions to deal with dates, times, and timezones.  

One way to define a datetime object is by specifying five numbers for the year, month, day, hour, and minute, in that order. Here are the first two rows of the data table:

In [ ]:
import datetime

t1 = datetime.datetime(2018,5,4,15,2)
t2 = datetime.datetime(2018,5,4,15,20)

# t1 and t2 are special python objects called "datetime" objects.
print(type(t1))
# If you print them with the notebook's "display" function, they're clearly 
# shown as datetime objects.
display(t1)
# If you print them with the "print" function, you get a string based on the standard
# way to print a date and time in the language your computer is set to use.
print(t1)

In [ ]:
# The datetime module also defines "date objects" and "time objects" that contain either 
# the date part of the time part of a datetime object. Here is how to access them. Mind
# the parentheses.
display(t1.date())
display(t1.time())
# Again, "display" shows them in a way that makes it clear that they're special objects, 
# whereas "print" shows them in a more human-readable way:
print()
print(t1.date())
print(t1.time())

#### The formating minilanguage

In the case of the wolf dataset, extracting the year number, month number, etc, from the date string and the time string would be tedious. Fortunately there's another method designed specifically to read a date and time off a string. To do that we need to specify the format of the date/time string using a formatting minilanguage outlined [here](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior):

In [ ]:
datetime_string = '5/4/18  15:02'
datetime_format = '%m/%d/%y  %H:%M'
# Each % followed by a letter is a placeholder for a specific part of the date/time.
# %m stands for the month (as a number), %y stands for the (2-digit) year, etc.
t1 = datetime.datetime.strptime(datetime_string,datetime_format)
display(t1)

You can also use the formatting language with the `strftime` function to print a datetime object in the format of your choice:

In [ ]:
datetime_format = 'Point taken on %A %B %d %Y at %H:%M.'
# %a stands for the name of the day of the week.
# %B stands for the full name of the month.
print(datetime.datetime.strftime(t1,datetime_format))

### 3.2. Datetime arithmetic

A crucial feature of datetime objects is that you can subtract them. The result is a new type of object called `timedelta`.

In [ ]:
dt = t2-t1
print(type(dt))
print(dt)

The reason this needs to be a new type of object distinct from datetime objects is that it makes sense to add two time differences (1 second + 2 seconds = 3 seconds), but it doesn't make sense to add two dates (June 1st + August 18th = ???). The same goes with multiplication: 3 x 3 seconds = 9 seconds, but 3 x January 10th = ???.  

To use a time difference in a formula like speed=distance/time, we'll first need to convert it to a number of seconds:

In [ ]:
print(dt.total_seconds())
# This is now a regular number that you can add/subtract/multiply/divide/etc.
# To get the time difference in a different unit (minutes, hours), first use
# total_seconds(), then convert (e.g., divide by 60 for minutes).

## 4. Wolf Distance Statistics (per day, month and hour)

### 4.1. Return of the daily mileage

Using the concepts learned in previous sections answer task 2. 

<div class="alert alert-block alert-danger">
<b>Task 2:</b>
Build a list of datetime objects corresponding to each row of the wolf V062 data table. Similar to task 1b, start with an empty list, then loop over the rows of the table. At each row, retrieve the date and time strings, convert them to a datetime object, and add that datetime object to the list. Once you're done, save that list as a new column of the table. Call the column "datetime".
</div>

*Hint: You can use `len(data)` to get the number of rows of the table.*

<div class="alert alert-block alert-danger">
<b>Task 3:</b>
Make the "date" column contain date objects rather than strings. Then, remake the daily mileage plot. This time pyplot should treat the dates as dates, not just as strings.
</div>

*Hint: Build a list of dates by looping over the rows of the "datetime" column and extracting each datetime object's date part. Use the resulting list to overwrite the "date" column.*

The daily mileage graph suggests the wolf travels more at some times of the year than other. To analyze this, let's plot the monthly mileage.

### 4.2. Monthly variations

Here is how to get the month of a datetime object:

In [ ]:
t1 = data['datetime'][0]
print(t1)
print(t1.month)

<div class="alert alert-block alert-danger">
<b>Task 4:</b>
Add a "month" column to the data table. Then, use `groupby` to compute and plot the distance traveled each month.
</div>

*Hint: Build a list of months by looping over the rows of the "datetime" column and extracting each datetime object's month. Use this list to create a new "month" column.*

### 4.3. Variations through the day

We can use the same approach to compute group the the total distance traveled during each hour of the day to see when the wolf is most active on average. Here is how to extract the hour of a datetime object:

In [ ]:
t1 = data['datetime'][0]
print(t1)
print(t1.hour)

<div class="alert alert-block alert-danger">
<b>Task 5:</b>
Repeat task 4 but group by hour of the day. When is the wolf most active on average?
</div>

**Note:** A weakness of the approach above is that all the distance traveled between times t1 and t2 is considered to "belong" to the hour of t1. For example, if the wolf travels one mile between 11:50am and 12:20pm, we add that mile to the "11am to 12pm" total. A better approach would be to add one third of it to the 11am-12pm total and two thirds to the 12pm-1pm total. It would take a fair bit more effort than what we did, but it's very achievable with the tools we've discussed in this class.

**Note:** While we're talking about the limitations of our analysis, it's very significant that we only have a point about every 20 minutes. A lot can happen in 20 minutes. The wolf could walk 100 yards and come back and we would never know about it. When interpreting the data it's always important to think about the data we don't have as well as the one we do have.

## 5. Traveling speed &ndash; How fast does the wolf move?

Another piece of information that can be easily extracted from the wolf data set is the animal's speed. Again, with one point about every 20 minutes this is going to be an averaged speed -- we're not going to measure any sprinting speed. Still, it's an important metric in quantifying the animal's motion pattern and estimating its energy expanditure.

To keep the original resolution of the data set, we want to compute the speed between every pair of consecutive points. We already have the distance between every pair of consecutive points. The other half of the equation is the time elapsed between the two points of each pair. Since we converted the dates and times to datetime objects, any two of which can be subtracted to get the time elapsed between them, this is going to work very much like the distance computation. Also like the distance computation, the first point doesn't have a previous point, thus no time elapsed since the previous point. We'll just use 0 instead. In the speed calculation the division by 0 will result in undefined behavior and pandas will just ignore that row.

<div class="alert alert-block alert-danger">
<b>Task 6:</b>
Compute the number of seconds between every pair of consecutive points and store it in a new column called "seconds since last point". Then, divide the "distance" column by the "seconds since last point" column and convert to miles per hour to create a new "speed" column.
</div>

Once you've created the "seconds since the previous point" column, getting the average speed since the previous point is just a matter of dividing the "distance" column by the "seconds since the previous point" column. Let's save the result in a new "speed" column.

In [ ]:
# Compute the speed between every pair of consecutive points.
data['speed'] = data['distance']/data['seconds since last point']*3600

A key advantage of the speed between consecutive points over the distance between the same points is that it doesn't make much sense to compare the distance traveled over a 20 minute interval with the one traveled over a 40 minute interval. If the tracking worked perfectly, on time, 100% of the time, this wouldn't matter, but real-world data almost always has gaps in it, this one included. Since the speed is the distance traveled per unit time, it does make sense to compare it across different time intervals.  

For example, we can make a box plot of the speed for each month of the year, or each hour of the day.

In [ ]:
# Make a boxplot of the speed for each month of the year.
data.boxplot(column='speed',by='month',grid=False,figsize=(12,8))
plt.suptitle('')
plt.title('"Instantaneous" speed throughout the year')
plt.xlabel('Month')
plt.ylabel('Speed (mph)')
plt.show()

In [ ]:
# The highest outliers dwarf the boxes themselves. Let's zoom in to see the boxes.
# Make a boxplot of the speed for each hour of the day.
data.boxplot(column='speed',by='month',grid=False,figsize=(12,8))
plt.suptitle('')
plt.title('"Instantaneous" speed throughout the year')
plt.xlabel('Month')
plt.ylabel('Speed (mph)')
plt.ylim(0,1) # This is where we zoom in.
plt.show()

Now the speed vs hour of the day:

In [ ]:
# Make a boxplot of the speed for each hour of the day.
data.boxplot(column='speed',by='hour',grid=False,figsize=(12,8))
plt.suptitle('')
plt.title('"Instantaneous" speed throughout the day')
plt.xlabel('Hour of the day')
plt.ylabel('Speed (mph)')
plt.ylim(0,2)
plt.show()

## 6. Extra: What about the other wolves?

<div class="alert alert-block alert-warning" style="background-color:#f0ddcc; color:#663300">
<b>Extra Task 1</b>  
So far we've only used the data from wolf "V068". How different would our results be if we'd chosen a different wolf? Can we quantify the behavior of an average wolf and the magnitude of the fluctuations from wolf to wolf?

<br/>  
Revisit one or more of the graphs we made in this notebook to address some of those questions.
</div>